In [95]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import pickle
import sklearn as sk
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from munkres import Munkres
import pandas as pd
from ktools import *
import igraph as ig
import leidenalg
import tqdm
from collections import defaultdict
import dill
import itertools

In [96]:
res = {}

In [97]:
def create_adjacency(df:pd.DataFrame)->np.array:
    m = max(max(df["zero"].value_counts().index)+ 1, max(df["one"].value_counts().index)) + 1
    A = np.zeros((m,m))
    a,b = df["zero"].to_numpy(dtype=int), df["one"].to_numpy(dtype=int)
 
    A[a,b] =1
            
    return A

In [98]:
data = pd.read_csv(r"data\email-Eu-core-temporal.txt.gz")
data = data.map(str.split)
series_d, series_0,  series_1, t = data.map(lambda x:(x[0], x[1])).values.flatten(),data.map(lambda x:x[0]).values.flatten(), data.map(lambda x:x[1]).values.flatten(), data.map(lambda x:x[2]).values.flatten()

series_d = pd.Series(series_d)
wt = pd.Series(series_d).value_counts()


to_integer = np.vectorize(int)
edges = pd.DataFrame({"zero": to_integer(series_0), "one": to_integer(series_1), "d": series_d, "w": series_d.map(lambda x: wt[x])})


edges

,zero,one,d,w
0,168,472,"(168, 472)",668
1,168,912,"(168, 912)",1212
2,2,790,"(2, 790)",229
3,2,322,"(2, 322)",10
4,2,790,"(2, 790)",229
...,...,...,...,...
332328,324,367,"(324, 367)",1
332329,790,126,"(790, 126)",150
332330,790,126,"(790, 126)",150
332331,782,126,"(782, 126)",54


In [99]:
#handling the edges
data = pd.read_csv(r"data\email-Eu-core.txt.gz")
data = data.map(str.split)
series_0 = data.map(lambda x:x[0]).values.flatten()
series_1 = data.map(lambda x:x[1]).values.flatten()

to_integer = np.vectorize(int)
series_0 = to_integer(series_0)


df = pd.DataFrame({"zero": to_integer(series_0), "one": to_integer(series_1)})
df = df.dropna()
df

,zero,one
0,2,3
1,2,4
2,5,6
3,5,7
4,8,9
...,...,...
25565,420,143
25566,174,859
25567,440,460
25568,52,786


In [100]:
# Handling the Labels
labels = pd.read_csv(r"data\email-Eu-core-department-labels.txt.gz")

labels= labels.map(str.split)
series_0 = labels.map(lambda x:x[0]).values.flatten() 
series_1 = labels.map(lambda x:x[1]).values.flatten()

to_integer = np.vectorize(int)
series_0 = to_integer(series_0)
labels = pd.DataFrame({"node": to_integer(series_0), "label": to_integer(series_1)})
labels = labels.dropna()



In [101]:
comms = {label: list(labels[labels["label"] == label]["node"]) for label in labels["label"].unique()}
labels["community"] = labels["label"].map(comms)



In [102]:
A = create_adjacency(df)
eucore = nx.from_numpy_array(A)
nx.set_node_attributes(eucore, labels["community"], "community")
nx.set_node_attributes(eucore, labels["label"], "label")
eucore.remove_node(1004)

In [103]:
c = {frozenset(eucore.nodes[c]["community"]) for c in eucore.nodes}#

c2 = set()
for community in c:
    if 1004 not in community:
        c2.add(community)
    else:
        x = list(community)
        x.remove(1004)
        c2.add(frozenset(x))


c = list(c2)


In [104]:
def modularity(A, partitions:list[int])->float:
    """returns the in degree of a given community/partition

    Args:
        partition (int): a list of nodes in the partition 

    Returns:
        float: in degree/weight of a community

    We know directed modularity can be calculated as:

    Q^D = 1/m sum_ij(Aij - (ki_in*kj_out)/m)delta(i,j)
        
    """

    m = np.sum(A)
    res = 0

    
    for partition in partitions:
        for i in partition:
            for j in partition:
                    ki = np.sum(A[:,i]) #+ np.sum(self.A[:,i])  # degree of element i
                    kj = np.sum(A[j,:]) #+ np.sum(self.A[:,j]) # degree of element j
                    a_ij = A[i,j] #gets the edge from i to j
                    mod = a_ij - ((ki*kj)/(m))
                    res += mod

    return res/m

In [108]:
modularity(nx.to_numpy_array(eucore),c2)

0.29999818295402103